# Clasificación de Sentimientos con una Red Neuronal Multicapa (MLP)

## Objetivo

En esta actividad vas a construir una red neuronal feedforward multicapa (MLP) usando PyTorch. El objetivo es entrenarla para que pueda clasificar frases en español como positivas o negativas.

### Con esto vas a:

- Comprender cómo se construye una red con múltiples capas y neuronas
- Usar funciones de activación no lineales (ReLU, Sigmoid)
- Implementar entrenamiento automático con optimizadores modernos
- Observar cómo una MLP mejora respecto al perceptrón simple del laboratorio anterior

### ¿Qué es una red neuronal multicapa?

A diferencia del perceptrón simple (una sola neurona), una MLP tiene:
- **Capa de entrada**: Recibe los features del texto
- **Capas ocultas**: Una o más capas intermedias que aprenden representaciones complejas
- **Capa de salida**: Produce la predicción final

Las capas ocultas permiten aprender patrones **no lineales**, lo que le da mucha más capacidad expresiva al modelo.

## 1. Preparación del entorno

Importamos PyTorch y NumPy para comenzar.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

print(f"PyTorch versión: {torch.__version__}")
print(f"Device disponible: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

PyTorch versión: 2.8.0+cu126
Device disponible: CPU


## 2. Datos de entrenamiento

Usamos un conjunto más grande de frases típicas de opiniones escritas en Argentina, etiquetadas como positivas (1) o negativas (0).

Vamos a incluir casos más variados y complejos que en el laboratorio anterior.

In [2]:
frases = [
    "La verdad, este lugar está bárbaro. Muy recomendable",
    "Una porquería de servicio, nunca más vuelvo",
    "Me encantó la comida, aunque la música estaba muy fuerte",
    "El envío fue lento y el producto llegó dañado. Qué desastre",
    "Todo excelente. Atención de diez",
    "Qué estafa, me arrepiento de haber comprado",
    "Muy conforme con el resultado final",
    "No me gustó para nada la experiencia",
    "Superó mis expectativas, gracias",
    "No lo recomiendo, mala calidad"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])  # 1 = Positivo, 0 = Negativo

print(f"Total de frases: {len(frases)}")
print(f"Balance: {sum(etiquetas)} positivas, {len(etiquetas) - sum(etiquetas)} negativas\n")
print("Ejemplos:")
for i in range(3):
    sentimiento = "Positivo" if etiquetas[i] == 1 else "Negativo"
    print(f"  {i+1}. '{frases[i]}' → {sentimiento}")

Total de frases: 10
Balance: 5 positivas, 5 negativas

Ejemplos:
  1. 'La verdad, este lugar está bárbaro. Muy recomendable' → Positivo
  2. 'Una porquería de servicio, nunca más vuelvo' → Negativo
  3. 'Me encantó la comida, aunque la música estaba muy fuerte' → Positivo


## 3. Construcción del vocabulario

Definimos manualmente un vocabulario con palabras que suelen aparecer en frases de opinión con carga positiva o negativa.

En este caso expandimos el vocabulario para cubrir más términos comunes.

In [3]:
vocabulario = [
    "bárbaro", "recomendable", "porquería", "nunca", "encantó",
    "fuerte", "desastre", "excelente", "estafa", "arrepiento",
    "conforme", "gustó", "superó", "gracias", "recomiendo", "mala"
]

print(f"Vocabulario: {len(vocabulario)} palabras clave")
print(f"\nPalabras: {vocabulario}")

Vocabulario: 16 palabras clave

Palabras: ['bárbaro', 'recomendable', 'porquería', 'nunca', 'encantó', 'fuerte', 'desastre', 'excelente', 'estafa', 'arrepiento', 'conforme', 'gustó', 'superó', 'gracias', 'recomiendo', 'mala']


## 4. Preprocesamiento: vectorización de las frases

Seguimos usando bag-of-words como en el laboratorio anterior: cada frase se convierte en un vector binario que indica si contiene alguna de las palabras del vocabulario.

Luego convertimos estos vectores a tensores de PyTorch para poder usarlos con redes neuronales.

In [4]:
def vectorizar(frase, vocabulario):
    """
    Convierte una frase en un vector binario según el vocabulario.

    Args:
        frase: String con la frase a vectorizar
        vocabulario: Lista de palabras clave

    Returns:
        Array de numpy con 1s y 0s (float32 para PyTorch)
    """
    tokens = frase.lower().split()
    return np.array([1 if palabra in tokens else 0 for palabra in vocabulario], dtype=np.float32)

# Vectorizamos todas las frases
X_np = np.array([vectorizar(frase, vocabulario) for frase in frases], dtype=np.float32)
y_np = etiquetas.astype(np.float32).reshape(-1, 1)

# Convertimos a tensores de PyTorch
X = torch.tensor(X_np)
y = torch.tensor(y_np)

print("Datos preprocesados:")
print(f"  X shape: {X.shape} (frases × features)")
print(f"  y shape: {y.shape} (frases × 1)")
print(f"\nPrimera frase vectorizada: {X[0]}")
print(f"Etiqueta: {y[0].item()}")

Datos preprocesados:
  X shape: torch.Size([10, 16]) (frases × features)
  y shape: torch.Size([10, 1]) (frases × 1)

Primera frase vectorizada: tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Etiqueta: 1.0


## 5. Definición del modelo MLP

Vamos a crear un modelo simple con:
- **Capa de entrada**: Tamaño = cantidad de palabras en el vocabulario
- **Capa oculta**: 8 neuronas con activación ReLU
- **Capa de salida**: 1 neurona con activación Sigmoid (para clasificación binaria)

### ¿Por qué estas activaciones?

- **ReLU** (Rectified Linear Unit): `f(x) = max(0, x)` → Introduce no linealidad, permite aprender patrones complejos
- **Sigmoid**: `f(x) = 1 / (1 + e^(-x))` → Convierte la salida a un valor entre 0 y 1 (probabilidad)

In [5]:
input_size = len(vocabulario)
hidden_size = 8

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # Definimos la arquitectura secuencial
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # Capa oculta
            nn.ReLU(),                            # Activación no lineal
            nn.Linear(hidden_size, 1),            # Capa de salida
            nn.Sigmoid()                          # Activación para clasificación binaria
        )

    def forward(self, x):
        """Propagación hacia adelante (forward pass)"""
        return self.net(x)

# Creamos una instancia del modelo
modelo = MLP()

print("Arquitectura del modelo:")
print(modelo)
print(f"\nParámetros totales: {sum(p.numel() for p in modelo.parameters())}")

Arquitectura del modelo:
MLP(
  (net): Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

Parámetros totales: 145


## 6. Configuración del entrenamiento

Necesitamos definir dos componentes clave:

### Función de pérdida (Loss Function)

Usamos **Binary Cross Entropy (BCE)**: mide qué tan diferentes son las predicciones del modelo de las etiquetas reales. El objetivo del entrenamiento es minimizar esta pérdida.

Fórmula: `BCE = -[y·log(ŷ) + (1-y)·log(1-ŷ)]`

### Optimizador

Usamos **Adam**: un optimizador moderno que ajusta automáticamente la tasa de aprendizaje para cada parámetro. Es más eficiente que el ajuste manual que hicimos en el perceptrón.

In [6]:
# Función de pérdida
criterio = nn.BCELoss()  # Binary Cross Entropy Loss

# Optimizador
optimizador = optim.Adam(modelo.parameters(), lr=0.01)

print("Configuración de entrenamiento:")
print(f"  Loss function: Binary Cross Entropy")
print(f"  Optimizador: Adam")
print(f"  Learning rate: 0.01")

Configuración de entrenamiento:
  Loss function: Binary Cross Entropy
  Optimizador: Adam
  Learning rate: 0.01


## 7. Entrenamiento del modelo

Vamos a entrenar el modelo por varias épocas. En cada época:
1. Calculamos las predicciones (forward pass)
2. Calculamos la pérdida
3. Calculamos los gradientes (backpropagation)
4. Actualizamos los pesos (optimizer step)

Este proceso es automático gracias a PyTorch, a diferencia del ajuste manual que hicimos en el perceptrón.

In [9]:
epocas = 50

print("="*60)
print("INICIANDO ENTRENAMIENTO")
print("="*60)
print(f"Épocas: {epocas}\n")

for epoca in range(epocas):
    # Modo entrenamiento
    modelo.train()

    # Forward pass: calculamos predicciones
    salida = modelo(X)

    # Calculamos la pérdida
    loss = criterio(salida, y)

    # Backpropagation: calculamos gradientes
    optimizador.zero_grad()  # Limpiamos gradientes previos
    loss.backward()           # Calculamos nuevos gradientes

    # Actualizamos pesos
    optimizador.step()

    # Mostramos progreso cada 10 épocas
    if (epoca + 1) % 10 == 0:
        print(f"Época {epoca+1:3d}, Pérdida: {loss.item():.4f}")

print("\n" + "="*60)
print("ENTRENAMIENTO FINALIZADO")
print("="*60)

INICIANDO ENTRENAMIENTO
Épocas: 50

Época  10, Pérdida: 0.0073
Época  20, Pérdida: 0.0066
Época  30, Pérdida: 0.0060
Época  40, Pérdida: 0.0055
Época  50, Pérdida: 0.0050

ENTRENAMIENTO FINALIZADO


## 8. Análisis del entrenamiento

Observá cómo la pérdida disminuye con el tiempo. Esto indica que el modelo está aprendiendo a clasificar mejor las frases.

Una pérdida cercana a 0 significa que el modelo está muy confiado en sus predicciones correctas.

## 9. Evaluación con frases nuevas

Probamos la red con frases que no estaban en el entrenamiento, para ver cómo generaliza.

In [10]:
frases_prueba = [
    "No me gustó la atención, bastante mala",
    "Muy buena experiencia, todo excelente",
    "Una estafa total, no lo recomiendo",
    "Súper conforme con el servicio",
    "Nada que ver con lo prometido, una decepción"
]

print("="*60)
print("EVALUACIÓN EN FRASES NUEVAS")
print("="*60)

# Vectorizamos las frases de prueba
X_prueba_np = np.array([vectorizar(frase, vocabulario) for frase in frases_prueba], dtype=np.float32)
X_prueba = torch.tensor(X_prueba_np)

# Modo evaluación (desactiva dropout, batch norm, etc.)
modelo.eval()

# Predicción sin calcular gradientes (más eficiente)
with torch.no_grad():
    predicciones = modelo(X_prueba)

# Mostrar resultados
for i, (frase, pred) in enumerate(zip(frases_prueba, predicciones), 1):
    probabilidad = pred.item()
    clase = "Positivo" if probabilidad >= 0.5 else "Negativo"
    print(f"\nFrase {i}: '{frase}'")
    print(f"  Predicción: {clase} (probabilidad: {probabilidad:.2f})")

    # Indicador visual de confianza
    if probabilidad >= 0.8 or probabilidad <= 0.2:
        print(f"  Confianza: Alta")
    elif probabilidad >= 0.6 or probabilidad <= 0.4:
        print(f"  Confianza: Media")
    else:
        print(f"  Confianza: Baja (ambiguo)")

print("\n" + "="*60)

EVALUACIÓN EN FRASES NUEVAS

Frase 1: 'No me gustó la atención, bastante mala'
  Predicción: Negativo (probabilidad: 0.00)
  Confianza: Alta

Frase 2: 'Muy buena experiencia, todo excelente'
  Predicción: Positivo (probabilidad: 0.97)
  Confianza: Alta

Frase 3: 'Una estafa total, no lo recomiendo'
  Predicción: Positivo (probabilidad: 0.99)
  Confianza: Alta

Frase 4: 'Súper conforme con el servicio'
  Predicción: Positivo (probabilidad: 1.00)
  Confianza: Alta

Frase 5: 'Nada que ver con lo prometido, una decepción'
  Predicción: Positivo (probabilidad: 0.98)
  Confianza: Alta



## 10. Reflexión final

### ¿Qué aprendimos?

1. **Arquitectura multicapa**: Vimos cómo una red con capas ocultas puede aprender representaciones más complejas que un perceptrón simple.

2. **Activaciones no lineales**: ReLU permite que la red aprenda patrones no lineales, algo imposible con un perceptrón simple.

3. **Entrenamiento automático**: PyTorch maneja automáticamente el cálculo de gradientes (backpropagation) y la actualización de pesos, a diferencia del ajuste manual del perceptrón.

4. **Probabilidades vs decisiones binarias**: La salida Sigmoid nos da una probabilidad (0-1) en lugar de solo 0 o 1, lo que permite medir la confianza del modelo.

### Ventajas sobre el perceptrón simple

- Puede aprender patrones más complejos (no lineales)
- Mejor capacidad de generalización
- Optimización más eficiente con Adam
- Salida probabilística (más informativa)

### Limitaciones que aún persisten

1. **No considera el orden de las palabras**: Bag-of-words sigue sin capturar secuencias
2. **Vocabulario fijo**: Solo conoce palabras predefinidas
3. **Sin contexto global**: Cada palabra se procesa independientemente
4. **Dataset pequeño**: Con solo 10 ejemplos, la generalización es limitada

### ¿Qué sigue?

En la próxima actividad vamos a ver cómo las **redes recurrentes (LSTM)** pueden procesar secuencias de palabras manteniendo memoria del contexto. Esto nos va a permitir:

- Capturar el orden de las palabras
- Entender dependencias temporales
- Procesar frases de longitud variable
- Aprovechar embeddings de palabras

Las LSTM son el paso previo a entender los Transformers, que revolucionaron el NLP.